In [ ]:
%matplotlib inline
import pandas as pd
from rich import print
import numpy as np
import matplotlib.pyplot as plt
#from quickbin import bin2d
from scipy.stats import binned_statistic_2d
import pdr
from astropy.visualization import ZScaleInterval
from astropy import wcs as pywcs


In [ ]:
tbl1 = pd.read_parquet('data/e41145/e41145-nd-0-catalog.parquet')
img1 = pdr.read('data/e41145/e41145-nd-ffull-b00-image-r.fits')
plt.figure(figsize=(8,8))
plt.imshow(ZScaleInterval()(img1['CNT']),cmap="binary_r",origin="lower")

In [ ]:
tbl2 = pd.read_parquet('data/e03909/e03909-nd-7-catalog.parquet')
img2 = pdr.read('data/e03909/e03909-nd-ffull-b07-image-r.fits')
plt.figure(figsize=(8,8))
plt.imshow(ZScaleInterval()(img2['CNT']),cmap="binary_r",origin="lower")

In [ ]:
cat = pd.concat([tbl1,tbl2])
cat

In [ ]:
ix = np.where((np.isfinite(cat['NUV_MDL_SIGMA'])) & 
              (cat['NUV_MDL_SIGMA']) & (cat['NUV_MDL_SIGMA']) &
              (cat['NUV_SOFTEDGE_FLAG_A6']==0) & (cat['NUV_HARDEDGE_FLAG_A6']==0) &
              (cat['NUV_GHOST_FLAG_A6']==0) & (cat['NUV_HOTSPOT_FLAG_A6']==0))
print(f'{100*np.round(len(ix[0])/len(cat),3)}% sources retained')

In [ ]:
statistic,xedge,yedge,binnum= binned_statistic_2d(
    cat['RA'].iloc[ix].values,cat['DEC'].iloc[ix].values,
    cat['NUV_MDL_BKG_CPS'].iloc[ix].values,
    bins=50,
    statistic='mean')
plt.figure(figsize=(8,8))
plt.imshow(ZScaleInterval()(statistic),cmap="inferno")


In [ ]:
def angularSeparation(ra1, dec1, ra2, dec2):
    d2r = np.pi/180.
    ra2deg = 1./d2r

    d1 = dec1*d2r
    d2 = dec2*d2r

    r1 = ra1*d2r
    r2 = ra2*d2r

    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))

    return r*ra2deg

In [ ]:
asp = pd.read_parquet('/Users/cm/github/gPhoton2/gPhoton/aspect/boresight_nosplit.parquet')
ra,dec = (148.0946706183358,47.6280701512940)
sep = angularSeparation(ra,dec,
                        asp['ra0'].values,asp['dec0'].values)
ix = np.where(sep<0.5)
asp.iloc[ix]

In [ ]:
cone = angularSeparation(ra,dec,cat['RA'].values,cat['DEC'].values)
cat.iloc[np.where(cone<0.1)]

In [ ]:
hdr = img2['CNT_HEADER']
wcs = pywcs.WCS(naxis=hdr['NAXIS'])
wcs.wcs.cdelt = [hdr['CDELT1'],
                 hdr['CDELT2']]
wcs.wcs.ctype = [hdr['CTYPE1'],
                 hdr['CTYPE2']]
wcs.wcs.crpix = [hdr['CRPIX1'],
                 hdr['CRPIX2']]
wcs.wcs.crval = [hdr['CRVAL1'],
                 hdr['CRVAL2']]


In [ ]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection=wcs)
ax.imshow(ZScaleInterval()(img2['CNT']),cmap="binary_r",origin="lower")

In [ ]:
tyc1_pos = (148.0946706183358,47.6280701512940)
tyc2_pos = (148.0967477582728,47.6274034468028)
imgpos = wcs.wcs_world2pix([tyc1_pos],1) # set the origin to FITS standard
imgx,imgy = imgpos[0]
imsz = np.shape(img1['CNT'])[-2:]

# crop on the subframe
# noting that image coordinates and numpy coordinates are flipped
boxsz = 50
x1, x2, y1, y2 = (max(int(imgy - boxsz), 0),
                  min(int(imgy + boxsz), imsz[0]),
                  max(int(imgx - boxsz), 0),
                  min(int(imgx + boxsz), imsz[1]))
plt.plot([50],[50],'rx')
plt.title('TYC 4144-329-1')
plt.imshow(ZScaleInterval()(img2['CNT'][x1:x2, y1:y2]), origin="lower", cmap="Greys_r")

In [ ]:
# THESE ARE NOT THE POSITIONS OF THESE SOURCES!!!
tyc1_pos = (148.0946706183358,47.6280701512940)
tyc2_pos = (148.0967477582728,47.6274034468028)
imgpos = wcs.wcs_world2pix([tyc2_pos],1) # set the origin to FITS standard
imgx,imgy = imgpos[0]
imsz = np.shape(img1['CNT'])[-2:]

# crop on the subframe
# noting that image coordinates and numpy coordinates are flipped
boxsz = 50
x1, x2, y1, y2 = (max(int(imgy - boxsz), 0),
                  min(int(imgy + boxsz), imsz[0]),
                  max(int(imgx - boxsz), 0),
                  min(int(imgx + boxsz), imsz[1]))
plt.plot([50],[50],'rx')
plt.title('TYC 4144-329-2')
plt.imshow(ZScaleInterval()(img2['CNT'][x1:x2, y1:y2]), origin="lower", cmap="Greys_r")

In [ ]:
foo = pdr.read('/Users/cm/Downloads/MAST_2025-06-24T2036/GALEX/2841351488930840576/MISDR2_00522_0771-nd-int.fits.gz')
foo.keys()

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(ZScaleInterval()(foo['PRIMARY']),cmap="binary_r",origin="lower")